In [2]:
from langchain_ollama import ChatOllama
from llms import get_llama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_chroma import Chroma
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field

from config import Config

print(Config.OLLAMA_MODEL, Config.OLLAMA_CONTEXT_SIZE)

llama3.2 10240


In [5]:
from typing import List, Optional, Dict

class Recorrente(BaseModel):
    recorrente: str = Field(description="Nome do recorrente")
    advogados: List[str] = Field(description="Nome e OAB do advogado ou dos advogados do recorrente")

class Recorrido(BaseModel):
    recorrido: str = Field(description="Nome do recorrido")
    advogados: List[str] | None = Field(description="Nome e OAB do advogado ou dos advogados do recorrido")

class Cabecalho(BaseModel):
    data_do_julgamento: str | None = Field(description="Data do julgamento")
    numero_do_processo: str = Field(description="Numero do processo ou numero do recurso eleitoral")
    origem: str = Field(description="Origem do processo")
    relator: str = Field(description="Relator do processo")
    recorrentes: List[Recorrente] = Field(description="Lista de recorrentes do processo e seus respectivos advogados")
    recorridos: List[Recorrido] = Field(description="Lista de recorridos do processo e seus respectivos advogados, se houver algum")

# class Relatorio(BaseModel):

In [6]:
print(Cabecalho.model_json_schema())

{'$defs': {'Recorrente': {'properties': {'recorrente': {'description': 'Nome do recorrente', 'title': 'Recorrente', 'type': 'string'}, 'advogados': {'description': 'Nome e OAB do advogado ou dos advogados do recorrente', 'items': {'type': 'string'}, 'title': 'Advogados', 'type': 'array'}}, 'required': ['recorrente', 'advogados'], 'title': 'Recorrente', 'type': 'object'}, 'Recorrido': {'properties': {'recorrido': {'description': 'Nome do recorrido', 'title': 'Recorrido', 'type': 'string'}, 'advogados': {'anyOf': [{'items': {'type': 'string'}, 'type': 'array'}, {'type': 'null'}], 'description': 'Nome e OAB do advogado ou dos advogados do recorrido', 'title': 'Advogados'}}, 'required': ['recorrido', 'advogados'], 'title': 'Recorrido', 'type': 'object'}}, 'properties': {'data_do_julgamento': {'anyOf': [{'type': 'string'}, {'type': 'null'}], 'description': 'Data do julgamento', 'title': 'Data Do Julgamento'}, 'numero_do_processo': {'description': 'Numero do processo ou numero do recurso ele

In [21]:
def split_documents(file_path, page_start=None, page_end=None, chunk_size=Config.SPLITTER_CHUNK_SIZE, chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP):
    loader = PyMuPDFLoader(file_path=file_path)
    doc = loader.load()[page_start:page_end]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [8]:
def cabecalho(doc, host, results=None, lock=None):
    llm: ChatOllama = get_llama(host=host)
    
    parser = JsonOutputParser(pydantic_object=Cabecalho)
    cabecalho_prompt = PromptTemplate.from_template("""### Missão
        Extraia do texto somente as seguintes informações:
        
        * Data do julgamento
        * Número do recurso eleitoral
        * Origem
        * Relator(a)
        * Recorrente(s)
        * Advogado(s) do(s) recorrente(s)
        * Recorrido
        * Advogado(s) do recorrido (se houver)
        
        ### Contexto
        
        {context}
        
        ### Resposta em lista pontuada
    """)
    lista_chain = cabecalho_prompt | llm
    lista = lista_chain.invoke({'context': doc})
    pprint(lista)
    content = lista.content
    pprint(content)
    
    json_prompt = PromptTemplate.from_template(
        """### Missão
    Extraia as informações contidas no contexto, de acordo com as instruções. Responda apenas com as informações extraídas, sem incluir as instruções.
    O contexto contém o cabeçalho de um acórdão, onde há um ou mais recorrentes (cada um com um ou mais advogados), sendo cada um demarcado no texto por 'RECORRENTE: ', e um recorrido (que pode ou não ser representado por um ou mais advogados), demarcado no texto por 'RECORRIDO: '. Um recorrente nunca pode ser um recorrido, e vice-versa.

    ### Contexto
    {context}

    ### Formato
    {format_instructions}

    ### Informações estruturadas em JSON"""
    )
    json_chain = json_prompt | llm
    response = json_chain.invoke({'context': content, 'format_instructions': parser.get_format_instructions()})
    pprint(response)
    parsed = parser.parse(response.content)
    pprint(parsed)

    if results:
        with lock:
            results['cabecalho'] = parsed
    
    return parsed

In [9]:
def relatorio(doc, host, results=None, lock=None):
    llm: ChatOllama = get_llama(host=host)
    
    parser = JsonOutputParser(pydantic_object=Relatorio)
    cabecalho_prompt = PromptTemplate.from_template("""### Missão
        Extraia do texto somente as seguintes informações:
        
        * Data do julgamento
        * Número do recurso eleitoral
        * Origem
        * Relator(a)
        * Recorrente(s)
        * Advogado(s) do(s) recorrente(s)
        * Recorrido
        * Advogado(s) do recorrido (se houver)
        
        ### Contexto
        
        {context}
        
        ### Resposta em lista pontuada
    """)
    lista_chain = cabecalho_prompt | llm
    lista = lista_chain.invoke({'context': doc})
    pprint(lista)
    content = lista.content
    pprint(content)
    
    json_prompt = PromptTemplate.from_template(
        """### Missão
    Extraia as informações contidas no contexto, de acordo com as instruções. Responda apenas com as informações extraídas, sem incluir as instruções.
    O contexto contém o cabeçalho de um acórdão, onde há um ou mais recorrentes (cada um com um ou mais advogados), sendo cada um demarcado no texto por 'RECORRENTE: ', e um recorrido (que pode ou não ser representado por um ou mais advogados), demarcado no texto por 'RECORRIDO: '. Um recorrente nunca pode ser um recorrido, e vice-versa.

    ### Contexto
    {context}

    ### Formato
    {format_instructions}

    ### Informações estruturadas em JSON"""
    )
    json_chain = json_prompt | llm
    response = json_chain.invoke({'context': content, 'format_instructions': parser.get_format_instructions()})
    pprint(response)
    parsed = parser.parse(response.content)
    pprint(parsed)

    if results:
        with lock:
            results['cabecalho'] = parsed
    
    return parsed

In [10]:
from pprint import pprint
from langchain_ollama import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


def stuff(docs, host, results, key, prompt_template, output_model, parser_type = JsonOutputParser):
    parser = parser_type(pydantic_object=output_model)
    prompt = PromptTemplate(
        template=prompt_template,
        input_variables=["context"],
        partial_variables={"format_instructions": parser.get_format_instructions()}
    )

    print("PROMPT")
    pprint(prompt)
    
    llm: ChatOllama = get_llama(host=host)

    embeddings_model = OllamaEmbeddings(
        base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
        model=Config.OLLAMA_EMBEDDINGS_MODEL
    )
    vectorstore = Chroma(
        collection_name='acordaos',
        embedding_function=embeddings_model,
    )
    vectorstore.add_documents(documents=docs)
    retriever = vectorstore.as_retriever()

    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    print(f"calling {host}")

        
    result = retrieval_chain.invoke({'input': 'Comece.'})
    pprint(result)
    answer = result['answer']
    print('answer')
    pprint(answer)

    json_prompt = PromptTemplate.from_template("""### Missão
    Extraia as informações contidas no contexto, de acordo com as instruções. Responda apenas com as informações extraídas, sem incluir as instruções.
    O contexto contém o cabeçalho de um acórdão, onde há um ou mais recorrentes (cada um com um ou mais advogados), sendo cada um demarcado no texto por 'RECORRENTE: ', e um recorrido (que pode ou não ser representado por um ou mais advogados), demarcado no texto por 'RECORRIDO: '. Um recorrente nunca pode ser um recorrido, e vice-versa.

    ### Contexto
    {context}

    ### Formato
    {format_instructions}

    ### Informações estruturadas em JSON"""
    )
    json_chain = json_prompt | llm
    json_chain.invoke({'context': answer, 'format_instructions': parser.get_format_instructions()})
    
    parsed = parser.parse(answer)
    pprint(parsed)
    # result = format_chain.invoke({'context': result['output_text']})

    print('\a')
    print(f"\nHOST {host}:\n")

    # simplificar_prompt = PromptTemplate.from_template(
    #     '''
    #     Simplifique a linguagem usada no texto. Para isso, siga as seguintes diretrizes
    #     '''
    # )

    results[key]['response'] = result['answer']
    results[key]['prompt'] = [
        str(prompt),
    ]

    # history.save('mapreduce', '', result['text'])
    return result

In [38]:
from langchain_community.embeddings import HuggingFaceEmbeddings

doc = split_documents(file_path, start_page, end_page, chunk_size=256, chunk_overlap=0)
# embeddings_model = OllamaEmbeddings(
#     base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
#     model=Config.OLLAMA_EMBEDDINGS_MODEL
# )
embeddings_model = HuggingFaceEmbeddings(
    model_name='neuralmind/bert-base-portuguese-cased',
)
vectorstore = Chroma(
    collection_name='acordaos',
    embedding_function=embeddings_model,
)
vectorstore.add_documents(documents=doc)
retriever = vectorstore.as_retriever()

No sentence-transformers model found with name neuralmind/bert-base-portuguese-cased. Creating a new one with mean pooling.
/home/rafael-albuquerque/LIODS/repos/langchain_first/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [40]:
for document in retriever.invoke('RECORRENTE'):
    print(document.page_content)

Jingle de Campanha "Faz o i” reproduzido durante o evento. Incentivo ao público a fazer gestos com 
as mãos formando a letra "i". Pedido explícito de votos.
4. 
Assinado eletronicamente por: DANIEL CARVALHO CARNEIRO 04/06/2024 10:18:10
CARNEIRO
CARNEIRO 04/06/2024 10:18:10
2.


In [7]:
results = {
    "cabecalho": {},
    "relatorio": {},
    "voto": {},
    "decisao": {},
}

In [12]:
file_path = "documentos/acordaos/0600012-49_REl_28052024_1.pdf"

In [14]:
start_page = 0
end_page = 1
doc = split_documents(file_path, start_page, end_page)
pprint(doc)

[Document(metadata={'source': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'file_path': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'page': 0, 'total_pages': 12, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'creator': '', 'producer': 'iText® 7.1.16 ©2000-2021 iText Group NV (AGPL-version)', 'creationDate': "D:20240604121301-03'00'", 'modDate': "D:20240604121301-03'00'", 'trapped': ''}, page_content='PODER JUDICIÁRIO\nTRIBUNAL REGIONAL ELEITORAL DO CEARÁ\n \nRECURSO ELEITORAL N. 0600012-49.2024.6.06.0033\nORIGEM: CANINDÉ/CE\nRELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO\nRECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ\nADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,\nLIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731\nRECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL\n \n \nEMENTA: ELEIÇÕES 2024. RECURSO ELEITORAL. PRÉ CANDIDATO AO CARGO DE \nPREFEITO. PROPAGANDA ELEITORAL ANTECIPADA. EVENTO COM ESTRUTURA \nROBUSTA. C

In [10]:
cabecalho_prompt = """### Missão
Extraia do texto as seguintes informações:

* Data do julgamento
* Número do recurso eleitoral
* Origem
* Relator(a)
* Recorrente(s)
* Advogado(s) do(s) recorrente(s)
* Recorrido
* Advogado(s) do recorrido (se houver)

### Contexto

{context}

### Resposta em lista pontuada
"""

In [11]:
# result = stuff(doc, 0, results, 'cabecalho', cabecalho_prompt, Cabecalho)
result = cabecalho(doc, 0, results)

AIMessage(content='Aqui estão as informações extraídas do texto:\n\n* Data do julgamento: 04/06/2024\n* Número do recurso eleitoral: 0600012-49.2024.6.06.0033\n* Origem: Canindé/CE\n* Relator(a): Desembargador Eleitoral Daniel Carvalho Carneiro\n* Recorrente(s): Antônio Ilomar Vasconcelos Cruz\n* Advogado(s) do(s) recorrente(s): Francisco Jardel Rodrigues de Souza (OAB CE32787-A), Lidienira Cavalcante Mendonça Vieira (OAB CE0016731)\n* Recorrido: Ministério Público Eleitoral\n* Advogado(s) do recorrido (se houver): Não disponível', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2024-10-09T15:07:30.890453934Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 51829413676, 'load_duration': 7442376288, 'prompt_eval_count': 992, 'prompt_eval_duration': 26321969000, 'eval_count': 184, 'eval_duration': 18007652000}, id='run-f4d13ab7-0181-4c5e-b9bb-1abeaccc576a-0', usage_metadata={'input_tokens': 992, 'output

In [12]:
cabecalho_dict = result
cabecalho_dict

{'data_do_julgamento': '04/06/2024',
 'numero_do_recurso_eleitoral': '0600012-49.2024.6.06.0033',
 'origem': 'Canindé/CE',
 'relator': 'Desembargador Eleitoral Daniel Carvalho Carneiro',
 'recorrentes': [{'recorrente': 'Antônio Ilomar Vasconcelos Cruz',
   'advogados': ['Francisco Jardel Rodrigues de Souza (OAB CE32787-A)',
    'Lidienira Cavalcante Mendonça Vieira (OAB CE0016731)']}],
 'recorridos': [{'recorrido': 'Ministério Público Eleitoral',
   'advogados': None}]}

In [54]:
start_page = 2
end_page = 5
doc = split_documents(file_path, start_page, end_page)

In [55]:
relatorio_prompt = PromptTemplate.from_template(
    """
    ### SISTEMA
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.

    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ```
    **2. Relatório (O Caso)**

    Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    O QUE DEVE CONTER EM CADA PARÁGRAFO:

    2.1. Parágrafo 1: Descreva as informações do processo inicial analisado pelo juiz, indicando:

    ◦ **O que o autor pediu?**

    **◦ O que o réu alegou para se defender?**

    2.2. Parágrafo 2: Apresente a decisão do juiz no processo inicial, descrevendo o que o juiz decidiu e as justificativas legais usadas, Cite artigos de lei e fundamentos jurídicos relevantes, considerando:

    ◦ **O que o juiz decidiu?**

    **◦ Quais foram as justificativas legais da decisão?**

    **◦ Cite artigos de lei e fundamentos jurídicos relevantes**

    2.3. Parágrafo 3: Descreva de forma simples quem recorreu e o que alegou, indicando:

    ◦ **Quem recorreu?**

    **◦ O que alegou para recorrer?**

    2.4. Parágrafo 4: Caixa de Texto Explicativa com Termos Jurídicos Relevante para compreensão do assunto principal

    ◦ **A caixa de texto explicativa tem o objetivo de fornecer definições e explicações simples de termos, expressões ou assuntos jurídicos relevantes para a compreensão do assunto principal do acórdão, ajudando o leitor a entender o contexto e o conteúdo legal envolvido.**

    **◦ Exemplo: “Propaganda antecipada negativa: A propaganda eleitoral antecipada negativa acontece quando, antes de 16 de agosto do ano eleitoral (art. 36 da Lei nº 9.504/1997), alguém faz críticas para prejudicar adversários políticos e influenciar eleitores. Essa prática é proibida e pode resultar em multa”.**
    ```

    ### CONTEXTO

    {context}

    ### RESPOSTA
    """
)

In [56]:
result = stuff(doc, 0, results, 'relatorio', relatorio_prompt)

calling 0
{'answer': '**2. Relatório (O Caso)**\n'
           '\n'
           '### O que o autor pediu?\n'
           '\n'
           'O autor, Antônio Ilomar Vasconcelos Cruz, recorreu contra a '
           'decisão do Tribunal Regional Eleitoral do Ceará, que havia negado '
           'seu pedido de reforma da sentença e julgamento improcedente da '
           'Representação. Ele argumentou que não realizou propaganda '
           'eleitoral ou conduta que possa macular o princípio de isonomia '
           'entre os candidatos.\n'
           '\n'
           '### O que o réu alegou para se defender?\n'
           '\n'
           'A Promotoria Eleitoral, representada pelo Procurador Regional '
           'Eleitoral Samuel Miranda Arruda, pugnou pelo não provimento do '
           'recurso. Ela argumentou que o recorrente realizou propaganda '
           'irregular, com pedido explícito de votos, e que a infraestrutura '
           'do evento e o local onde fora realizado seriam indicat

In [57]:
start_page = 5
end_page = 11
doc = split_documents(file_path, start_page, end_page)

In [58]:
voto_prompt = PromptTemplate.from_template(
    """
    ### SISTEMA
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.

    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ```
    ### 3. Voto (Argumentação/ Motivação/ Fundamentação)

    Apresente as razões jurídicas que embasaram a decisão do Tribunal, destacando, em 2 parágrafos, com no máximo 3 linhas, de texto contínuo, considerando:
    
    ◦ **Quais foram os princípios legais aplicados.**
    
    **◦ A interpretação dada à legislação.**
    
    **◦ Argumentos relevantes usados na motivação.**
    
    **◦ Use uma linguagem simples e evite jargões técnicos sempre que possível.**
    
    O QUE DEVE CONTER EM CADA PARÁGRAFO:
    
    3.1. Parágrafo 1: Fundamentação de Fato e de Direito: 
    
    ◦ **Explique os fatos relevantes.**
    
    **◦ Apresente a fundamentação jurídica, citando artigos de lei, precedentes jurisprudenciais que embasam a decisão do Tribunal.**
    
    3.2. Parágrafo 2: Conclusão e Decisão:
    
    ◦ **Conclusão do Voto.**
    
    **◦ Decisão Final com possíveis implicações.**
    ```

    ### CONTEXTO

    {context}

    ### RESPOSTA
    """
)

In [59]:
result = stuff(doc, 0, results, 'voto', voto_prompt)

calling 0
{'answer': '### 3. Voto (Argumentação/Motivação/Fundamentação)\n'
           '\n'
           '#### 3.1. Parágrafo 1: Fundamentação de Fato e de Direito:\n'
           '\n'
           'O caso em julgamento envolveu a realização de propaganda eleitoral '
           'antecipada por parte do representado, que fazia expressa alusão à '
           'sua candidatura e pedido explícito de votos. Os documentos '
           'colacionados à inicial demonstram claramente essa conduta.\n'
           '\n'
           'A fundamentação jurídica é baseada na Lei 9.504/97, que regula as '
           'eleições diretas para prefeito. O artigo 36-A desse diploma legal '
           'estabelece que a propaganda eleitoral só pode ser realizada '
           'durante o período oficial de campanha. Além disso, o artigo 36, § '
           '3º, prevê penalidade para atos não abrangidos pelo rol do artigo '
           '36-A.\n'
           '\n'
           '#### 3.2. Parágrafo 2: Conclusão e Decisão:\n'
     

In [60]:
start_page = 10
end_page = 12
doc = split_documents(file_path, start_page, end_page)

In [61]:
decisao_prompt = PromptTemplate.from_template(
    """
    ### SISTEMA
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.

    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ```
    ### **4. Resultado do Julgamento (Dispositivo):**

    Informe de maneira direta e clara o resultado final da decisão, incluindo em 2 parágrafos, com no máximo 3 linhas em cada um, em texto contínuo:
    
    4.1. Parágrafo 1: Resultado do Julgamento
    
    Aqui, deverá ser informado o resultado final da decisão, se a decisão foi unânime ou não, se manteve a decisão original ou a modificou.
    
    **◦ Resuma o resultado do julgamento de forma clara e objetiva.**
    
    **◦ Exemplo: “O Tribunal Regional Eleitoral do Ceará, por unanimidade, decidiu manter a decisão original, entendendo que...".**
    
    4.2. Parágrafo 2: Decisão Clara e Direta
    
    **◦ Indique de forma concisa apenas uma das opções considerando o PDF ou texto inserido:
    - Recurso Negado;
    - Recurso Aceito;
    - Recurso Parcialmente Aceito.**
    
    **◦ Exemplo: "Recurso Negado."**
    ```

    ### CONTEXTO

    {context}
    """
)

In [62]:
result = stuff(doc, 0, results, 'decisao', decisao_prompt)

calling 0
{'answer': '### **4. Resultado do Julgamento (Dispositivo):**\n'
           '\n'
           '#### Parágrafo 1: Resultado do Julgamento\n'
           '\n'
           'O Tribunal Regional Eleitoral do Ceará, por unanimidade, decidiu '
           'manter a decisão original, entendendo que a utilização de jingle '
           'de campanha faz expressa alusão à candidatura e extrai pedido '
           'explícito de votos.\n'
           '\n'
           '#### Parágrafo 2: Decisão Clara e Direta\n'
           '\n'
           'Recurso Negado.',
 'context': [Document(metadata={'author': '', 'creationDate': "D:20240604121301-03'00'", 'creator': '', 'file_path': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20240604121301-03'00'", 'page': 2, 'producer': 'iText® 7.1.16 ©2000-2021 iText Group NV (AGPL-version)', 'source': 'documentos/acordaos/0600012-49_REl_28052024_1.pdf', 'subject': '', 'title': '', 'total_pages': 12, 'trapped': ''

In [63]:
prompts = f"""# Cabeçalho

{results['cabecalho']['prompt']}

# Relatório

{results['relatorio']['prompt']}

# Voto

{results['voto']['prompt']}

# Decisão

{results['decisao']['prompt']}
"""
print(len(prompts))
pprint(prompts)

7533
('# Cabeçalho\n'
 '\n'
 '["input_variables=[\'context\'] input_types={} partial_variables={} '
 "template='\\\\n    ### Sistema\\\\n\\\\n    Extraia do texto somente as "
 'seguintes informações:\\\\n    \\\\n    -   Número do processo (ou número do '
 'recurso eleitoral)\\\\n    -   Origem\\\\n    -   Relator\\\\n    -   '
 'Recorrente\\\\n    -   Advogados(as)\\\\n    -   Recorrido\\\\n    -   '
 'Assunto\\\\n    \\\\n    Formate sua resposta como uma lista, assim:\\\\n    '
 '\\\\n    -   Número do processo: Número do processo\\\\n    -   Origem: '
 'Origem\\\\n    -   Relator: Relator\\\\n    -   Recorrente: '
 'Recorrente\\\\n    -   Advogados: Advogados(as)\\\\n    -   Recorrido: '
 'Recorrido\\\\n    -   Assunto: Assunto, em 2 ou 3 palavras\\\\n    \\\\n    '
 '### Exemplo de entrada:\\\\n    \\\\n    ```\\\\n    PODER '
 'JUDICIÁRIO\\\\n    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ\\\\n    \\\\n    '
 'RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033\\\\n    ORIGEM: '
 'CANINDÉ/C

In [64]:
acordao = f"""{results['cabecalho']['response']}

{results['relatorio']['response']}

{results['voto']['response']}

{results['decisao']['response']}
"""
print(len(acordao))
pprint(acordao)

4426
('-   Número do processo: 0600012-49.2024.6.06.0033\n'
 '-   Origem: CANINDÉ/CE\n'
 '-   Relator: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO\n'
 '-   Recorrente: ANTÔNIO ILOMAR VASCONCELOS CRUZ\n'
 '-   Advogados(as): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A, '
 'LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731\n'
 '-   Recorrido: MINISTÉRIO PÚBLICO ELEITORAL\n'
 '-   Assunto: Propaganda Eleitoral\n'
 '\n'
 '**2. Relatório (O Caso)**\n'
 '\n'
 '### O que o autor pediu?\n'
 '\n'
 'O autor, Antônio Ilomar Vasconcelos Cruz, recorreu contra a decisão do '
 'Tribunal Regional Eleitoral do Ceará, que havia negado seu pedido de reforma '
 'da sentença e julgamento improcedente da Representação. Ele argumentou que '
 'não realizou propaganda eleitoral ou conduta que possa macular o princípio '
 'de isonomia entre os candidatos.\n'
 '\n'
 '### O que o réu alegou para se defender?\n'
 '\n'
 'A Promotoria Eleitoral, representada pelo Procurador Regional Eleitoral '
 'Samu

In [69]:
simplificar_prompt = PromptTemplate.from_template("""
    Simplifique o texto, mantendo sua estrutura original e seguindo as seguintes diretivas:

    1. Utilize sempre uma linguagem simples e direta, evitando termos técnicos desnecessários. Caso seja imprescindível usar um termo jurídico específico, certifique-se de explicá-lo claramente.
    2. Mantenha o número de parágrafos e linhas conforme indicado para cada seção.
    3. Assegure-se de que as informações jurídicas sejam precisas e que a fundamentação legal esteja diretamente relacionada ao caso em discussão.
    4. Siga as regras de linguagem simples: seja empático e cordial, use frases curtas, escreva na ordem direta e em voz ativa, coloque as informações mais importantes no início e utilize verbos de ação.
    5. Revise o texto para garantir que seja claro e acessível, evitando siglas, jargões e termos técnicos. Confirme se todas as diretrizes e restrições foram cumpridas.
    6. Adote uma escrita humanizada, procurando aproximar o texto da forma como um ser humano se expressaria, para tornar a comunicação mais próxima e compreensível.

    ### CONTEXTO

    ```
    {context}
    ```

    ### RESPOSTA
""")

In [70]:
simplificar_chain = simplificar_prompt | get_llama(0)

In [71]:
result = simplificar_chain.invoke({'context': acordao})

In [72]:
pprint(result.content)

('**Contexto**\n'
 '\n'
 'O caso em questão envolveu a realização de propaganda eleitoral antecipada '
 'por parte do representado, que fazia expressa alusão à sua candidatura e '
 'pedido explícito de votos. A fundamentação jurídica é baseada na Lei '
 '9.504/97, que regula as eleições diretas para prefeito.\n'
 '\n'
 '**O que o autor pediu?**\n'
 '\n'
 'O autor recorreu contra a decisão do Tribunal Regional Eleitoral do Ceará, '
 'argumentando que não realizou propaganda eleitoral ou conduta que possa '
 'macular o princípio de isonomia entre os candidatos.\n'
 '\n'
 '**O que o réu alegou para se defender?**\n'
 '\n'
 'A Promotoria Eleitoral pugnou pelo não provimento do recurso, argumentando '
 'que o recorrente realizou propaganda irregular e que a infraestrutura do '
 'evento e o local onde fora realizado seriam indicativos do objetivo de '
 'ampliar o alcance da mensagem de sua pré-candidatura.\n'
 '\n'
 '**O que o juiz decidiu?**\n'
 '\n'
 'O Tribunal Regional Eleitoral do Ceará